In [2]:
import sys
import pandas as pd
import numpy as np

sys.path

['/Users/mirandavincent/econ4161/econ4161-finalproject',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages']

In [183]:
df = pd.read_csv('401k.csv')
df

,ira,a401,hval,hmort,hequity,nifa,net_nifa,tfa,net_tfa,tfa_he,...,educ,db,marr,male,twoearn,e401,p401,pira,net_n401,hown
0,0,0,69000,60150,8850,100,-3300,100,-3300,5550,...,12,0,1,0,0,0,0,0,-3300,1
1,0,0,78000,20000,58000,61010,61010,61010,61010,119010,...,16,0,0,0,0,0,0,0,61010,1
2,1800,0,200000,15900,184100,7549,7049,9349,8849,192949,...,11,0,1,1,1,0,0,1,8849,1
3,0,0,0,0,0,2487,-6013,2487,-6013,-6013,...,15,0,1,0,1,0,0,0,-6013,0
4,0,0,300000,90000,210000,10625,-2375,10625,-2375,207625,...,12,1,0,0,0,0,0,0,-2375,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,0,172,0,0,0,0,0,172,172,172,...,14,1,0,0,0,1,1,0,0,0
9911,0,4500,53000,49000,4000,936,-3664,5436,836,4836,...,12,1,1,1,0,1,1,0,-3664,1
9912,0,0,250000,150000,100000,10150,6150,10150,6150,106150,...,16,0,1,0,1,1,0,0,6150,1
9913,0,15000,63000,59000,4000,3499,-501,18499,14499,18499,...,18,1,1,0,1,1,1,0,-501,1


In [171]:
df[df.loginc == float('-inf')]

,ira,a401,hval,hmort,hequity,nifa,net_nifa,tfa,net_tfa,tfa_he,...,db,marr,male,twoearn,e401,p401,pira,net_n401,hown,loginc
4508,0,0,58000,27000,31000,0,0,0,0,31000,...,0,1,0,0,0,0,0,0,1,-inf
5515,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,-inf
6895,0,0,30000,0,30000,20000,16000,20000,16000,46000,...,0,1,0,1,1,0,0,16000,1,-inf


In [184]:
# Generate log income column -- drop rows which are either minus infty or NaN.
df['loginc'] = np.log(df.inc)
df.drop(
  index=df[
    (df.loginc == float('-inf'))
    | pd.isna(df.loginc)
  ].index,
  inplace=True,
)

/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [193]:
_df = pd.DataFrame({
  'a': [1, 2, 3],
})
s = pd.Series([4, 5, 6])

_df = pd.concat([_df, s], axis=1)
_df

,a,0
0,1,4
1,2,5
2,3,6


In [201]:
controls = [
  'age', 'loginc', 'fsize', 'educ', 'marr', 'twoearn', 'db', 'pira', 
  'hown', 'ira', 'male', 'hmort', 'hequity', 'hval',
]

combinations = [
  # Generate combinations for all possible pairwise tuples, inclusive of index (e.g. to generate loginc^2).
  (controls[i], controls[j]) for i in range(0, len(controls)) for j in range(i, len(controls))
]
# Using a separate interaction columns list for convenient merging via pd.concat. Otherwise
# pd PerformanceWarnings get thrown.
interaction_columns = []

for _tuple in combinations:
  col_name = f'{_tuple[0]}^2' if _tuple[0] == _tuple[1] else f'{_tuple[0]}:{_tuple[1]}'
  controls.append(col_name)
  col = (df[_tuple[0]] * df[_tuple[1]]).rename(col_name)
  interaction_columns.append(col)

df = pd.concat(
  [df, *interaction_columns],
  axis=1,
)

print(f'Generated {len(combinations)} interactions.')
df

Generated 105 interactions.


,ira,a401,hval,hmort,hequity,nifa,net_nifa,tfa,net_tfa,tfa_he,...,male^2,male:hmort,male:hequity,male:hval,hmort^2,hmort:hequity,hmort:hval,hequity^2,hequity:hval,hval^2
0,0,0,69000,60150,8850,100,-3300,100,-3300,5550,...,0,0,0,0,3618022500,532327500,4150350000,78322500,610650000,4761000000
1,0,0,78000,20000,58000,61010,61010,61010,61010,119010,...,0,0,0,0,400000000,1160000000,1560000000,3364000000,4524000000,6084000000
2,1800,0,200000,15900,184100,7549,7049,9349,8849,192949,...,1,15900,184100,200000,252810000,2927190000,3180000000,33892810000,36820000000,40000000000
3,0,0,0,0,0,2487,-6013,2487,-6013,-6013,...,0,0,0,0,0,0,0,0,0,0
4,0,0,300000,90000,210000,10625,-2375,10625,-2375,207625,...,0,0,0,0,8100000000,18900000000,27000000000,44100000000,63000000000,90000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,0,172,0,0,0,0,0,172,172,172,...,0,0,0,0,0,0,0,0,0,0
9911,0,4500,53000,49000,4000,936,-3664,5436,836,4836,...,1,49000,4000,53000,2401000000,196000000,2597000000,16000000,212000000,2809000000
9912,0,0,250000,150000,100000,10150,6150,10150,6150,106150,...,0,0,0,0,22500000000,15000000000,37500000000,10000000000,25000000000,62500000000
9913,0,15000,63000,59000,4000,3499,-501,18499,14499,18499,...,0,0,0,0,3481000000,236000000,3717000000,16000000,252000000,3969000000


In [226]:
np.asarray(a > 2)

array([ True,  True,  True,  True,  True])

In [235]:
a = np.array([10, 20, 30, 40, 50])
b = (a > 2).nonzero()[0]
b

array([0, 1, 2, 3, 4])

In [252]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

# Add p401 to list of controls such that outcome feature selection via Lasso downgrades
# importance of features correlated with p401 (which will be recovered by second p401 ~ controls)
# regression.
X = df[controls + ['p401']]
y = df.tw

lasso_pipeline = make_pipeline(
  StandardScaler(), 
  LassoCV(
    cv=10, 
    random_state=0, 
    n_jobs=-1, 
    # Lift default tolerance to try to converge faster (this LassoCV uses coordinate descent to find best alpha)
    tol=1e-2 
  )
)
lasso_pipeline.fit(X, y)
lasso_coefs = lasso_pipeline['lassocv'].coef_
# Using .nonzero() is slightly preferable for subclass behavior.
selected_outcome_indices = np.asarray(np.abs(lasso_coefs) > 1e-6).nonzero()[0]
selected_outcome_vars = X.columns[selected_outcome_indices]

/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.749e+12, tolerance: 1.233e+12
  model = cd_fast.enet_coordinate_descent(


In [ ]:
from sklearn.linear_model import LogisticRegressionCV

# Remove P401 from list of controls (now outcome). See Kai notebook for details.
X2 = X.iloc[:, 1:-1]
y2 = df.p401

log_pipeline = make_pipeline(
  StandardScaler(), 
  LogisticRegressionCV(
    penalty='l1',
    cv=10,
    random_state=0,
    # Liblinear preferred for binary schemes (and small datasets). See 
    # See https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html.
    solver='liblinear',
    tol=1e-2
  )
)
log_pipeline.fit(X2, y2)
log_coefs = log_pipeline['logisticregressioncv'].coef_
selected_treatment_indices = np.asarray(np.abs(log_coefs[0]) > 1e-6).nonzero()
selected_treatment_vars = X2.columns[selected_treatment_indices]

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

union_of_selected_vars = list(set(selected_outcome_vars) | set(selected_treatment_vars))
X3 = X[union_of_selected_vars]

X_train, X_test, y_train, y_test = train_test_split(
  X3, 
  y,
  test_size=0.1,
  random_state=0
)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Run standard OLS with the selected covariates.
res = sm.OLS(y_train, X_train).fit()

y_pred = res.predict(X_test)
rmse = np.sqrt(((y_test - y_pred) ** 2).mean())

print(f'Coefficient on P401 was: {res.params.get("p401")}')

Coefficient on P401 was: 10841.238753606563


In [276]:
rmse

61598.75719280442